In [0]:
#Instalação das bibliotecas necessárias

from pyspark.sql import functions as F
from pyspark.sql.types import DateType, TimestampType

In [0]:
%sql
select * from tb_proposicoes_bronze;

In [0]:

df_proposicoes_bronze = spark.table("tb_proposicoes_bronze") # Carrega a tabela bronze

# Transformações
df_proposicoes_silver = (
     df_proposicoes_bronze
     .withColumn(
        "ementa",
        F.when(F.col("ementa").isNull(), F.lit("Não informado"))
        .otherwise(F.initcap(F.trim(F.col("ementa"))))
    )

    .withColumn(
        "siglaTipo",
        F.when(F.col("siglaTipo").isNull(), F.lit("Não se aplica"))
        .otherwise(F.trim(F.col("siglaTipo")))
    )

    .withColumn("uri", F.trim(F.col("uri")))
    .withColumn("url_origem", F.trim(F.col("url_origem")))

    .withColumn("data_ingestao_silver", F.current_timestamp()) # adiciona coluna de controle da camada silver
)




In [0]:
# Salva a tabela em Delta 
df_proposicoes_silver.write.format("delta").mode("overwrite").saveAsTable("tb_proposicoes_silver")

print("Tabela 'tb_proposicoes_silver' criada/atualizada com sucesso!")

In [0]:
%sql
select * from tb_proposicoes_silver;